# Diseño de un Marco de KRIs y Panel de Control
---

Aplicación del enfoque AMA (Advanced Measurement Approach) con modelo LDA (Loss Distribution Approach), integración de Indicadores Clave de Riesgo - (Key Risk Indicators - KRIs) y la Evaluación de la Adecuación del Capital Interno (ICAAP).

Como analista de riesgos, usted calculó el capital económico (OpVaR y OpES) para diferentes líneas de negocio (Ej. Banca Corporativa -'CB' (puede ser otra línea: 'PS', ...)) basado en datos o registro históricos de pérdidas.

Ahora, su rol cambia de analista a gestor de riesgos. Su tarea es diseñar un marco de monitoreo proactivo para las mismas líneas de negocio. Este marco debe identificar los riesgos antes de que se conviertan en pérdidas monetarias, utilizando **Indicadores Clave de Riesgo (KRIs)**.

EL objetivo es diseñar un sistema que nos permita:

1. Diferenciar entre la medición del desempeño (KPIs) y la medición del riesgo (KRIs).
2. Establecer umbrales de alerta para un KRI crítico.
3. Vincular este sistema de monitoreo con la gestión de capital y los reportes regulatorios.

**Estrategias de Mitigación**

* Fortalecer los controles internos.
* Implementar auditorías y revisiones periódicas.
* Implemnetar políticas de gestión de riesgos.


## 1. Escenario: Caso de estudio

Tras un análisis de la línea de negocio, usted identifica la siguiente información:

1. **KPI Principal**: El indicador clave de desempeño de la gerencia es el "Tiempo de Aprobación de Créditos". Actualmente, el objetivo es de 5 días hábiles.

2. **Contexto Operativo:** Para lograr esta meta de 5 días, se ha presionado al personal. Usted observa dos problemas emergentes:

* Personal: La tasa de rotación en el área de analistas de crédito de la línea ha aumentado un 15% en el último año.

* Sistemas: El software de análisis crediticio (CreditSys v2.0) tiene 8 años de antigüedad y sufre caídas frecuentes (reportadas en TI) y requiere muchos "parches" manuales en Excel por parte de los analistas para finalizar las solicitudes.

El modelo LDA le mostró que las pérdidas históricas de la línea son infrecuentes pero muy costosas, usualmente ligadas a errores en la originación de créditos (un evento de "Ejecución, Entrega y Gestión de Procesos").

## 2. Tareas a Desarrollar

**Parte 1: Identificación de KPIs vs. KRIs**

**1. Analice el KPI**: "Tiempo de Aprobación de Créditos".

¿Por qué es un indicador de desempeño?

**2. Derive los KRIs**: Basado en el contexto operativo, se proponen tres (3) KRIs que alertarían de forma activa sobre un aumento en el riesgo operacional.

* KRI de Personas: (Ej. relacionado con la rotación, capacitación, horas extra).
* KRI de Procesos: (Ej. relacionado con errores, retrabajos, uso de Excel).
* KRI de Sistemas: (Ej. relacionado con caídas del sistema, tickets de soporte).

Cada KRI propuesto se relaciona directamente con el KPI principal.

**Tabla comparativa**:

| Indicador | Tipo (KPI/KRI) | ¿Qué Mide? (Desempeño/Riesgo) | Justificación (Por qué es KRI/KPI) |
| --- | --- | --- | --- |
| Tiempo de Aprobación de Créditos | KPI | Desempeño (Eficiencia) | Mide un resultado pasado (éxito del proceso) |
| [KRI de Personas] | KRI | Riesgo (Prospectivo) | Ej. Alta rotación predice más errores... |
| [KRI de Procesos] | KRI | Riesgo (Prospectivo) | Ej. Aumento de ""parches"" manuales... |
| [KRI de Sistemas] | KRI | Riesgo (Prospectivo) | Ej. Más caídas del sistema predicen... |


**Categorias tratadas**:

* Tasa de Rotación (Personas)
* Parches Manuales (Procesos)
* Caídas del Sistema (Sistemas)

## Parte 2: Diseño de Panel de Control (Dashboard)
---

El monitoreo de KRIs requiere umbrales claros. Elija uno (1) de los KRIs que propuso (ej. "Tasa de Error Manual en Originación" o "% de Caídas del Sistema").

**1. Definición de Umbrales**

Diseñe un sistema de "semáforo" (RAG - Red, Amber, Green) para este KRI. Defina los umbrales numéricos para cada nivel:

* Verde (Aceptable): El nivel normal de operación.
* Ámbar (Advertencia / Tolerable): El riesgo está aumentando y requiere monitoreo cercano.
* Rojo (Acción Requerida): El riesgo ha superado el apetito y requiere acción inmediata.

**2. Justificación de Umbrales**

¿Cómo establecería estos umbrales en la práctica? (Ej. ¿basado en promedios históricos, en el apetito de riesgo definido por la junta, en benchmarks de la industria?).

**3. Diseño Visual**

Describa (o dibuje esquemáticamente) cómo se vería este KRI en un panel de control gerencial. (Ej. un "velocímetro", un gráfico de serie de tiempo con las bandas de colores, etc.).

## Parte 3: Vínculo con Reportes Regulatorios y Capital
---

Esta es la conexión crítica entre el monitoreo (KRIs) y la cuantificación (LDA).

**Escenario de Estrés**

Imagine que su KRI de Sistemas ("% de Caídas del Sistema") y su KRI de Procesos ("% de "parches" manuales") han estado en 'Rojo' durante los últimos 4 meses.

Pregunta: Su modelo OpVaR  se basa en datos históricos de 10 años, donde nunca ha ocurrido un fallo catastrófico del sistema. El KRI en rojo le dice que el futuro podría no parecerse al pasado. ¿Cómo impacta este KRI en rojo su reporte de capital?


In [ ]:
import pandas as pd
import numpy as np
import datetime
import matplotlib.pyplot as plt
import scipy.stats as stats
import plotly.graph_objects as go
import plotly.express as px
import seaborn as sns
import matplotlib.ticker as ticker
from ipywidgets import interact, Dropdown
from IPython.display import display, Markdown, HTML
import warnings      # Manejo de warnings
warnings.filterwarnings('ignore')

In [ ]:
# Datos de pérdidas historicas
df_losses = pd.read_excel('dataloss.xlsx')

In [ ]:
# Función para ejecutar el modelo LDA
def lda_model(df_losses, line_of_business):
    """
    Ejecuta el modelo LDA completo (Monte Carlo: Freq, Sev) para una línea de negocio.
    """
    # 1. Filtrar y preparar datos y crear variable 'Net Loss'
    df_line = df_losses[df_losses['Business Line'] == line_of_business].copy()
    df_line['Net Loss'] = df_line['Gross Loss Amount'] - df_line['Recovery Amount']
    df_line = df_line[df_line['Net Loss'] > 0]

    # 2. Modelo de Frecuencia
    frequency_data = df_line.set_index('Date').resample('1Y')['ID'].count()
    lambda_anual = frequency_data.mean()

    # 3. Modelo de Severidad (modelar usando Log-Normal)
    severity_data = df_line['Net Loss']
    shape_sev, loc_sev, scale_sev = stats.lognorm.fit(severity_data, floc=0)

    # 4. Simulación Monte Carlo
    N_SIMS = 50_000            # Ajustar tamaño muestras MC para mayor presición
    total_annual_losses = []
    for _ in range(N_SIMS):
        num_events = stats.poisson.rvs(lambda_anual)
        if num_events > 0:
            losses = stats.lognorm.rvs(s=shape_sev, loc=loc_sev, scale=scale_sev, size=num_events)
            total_annual_losses.append(np.sum(losses))
        else:
            total_annual_losses.append(0)

    total_annual_losses = np.array(total_annual_losses)

    # 5. Calcular Capital
    op_var_999 = np.percentile(total_annual_losses, 99.9)
    op_es_999 = total_annual_losses[total_annual_losses > op_var_999].mean()

    return total_annual_losses, op_var_999, op_es_999, (lambda_anual, shape_sev, loc_sev, scale_sev)


**Datos consolidados**

In [ ]:
# DataFrames de la aplicación
print("\n Datos de pérdidas generados:")
display(df_losses.head())


 Datos de pérdidas generados:


,Unnamed: 0,ID,Date,Business Line,Risk Category,Gross Loss Amount,Recovery Amount
0,0,1,2012-10-31,CORP,FRAUD,1490.558633,0.000000
1,1,2,2010-12-07,CORP,EDPM,14129.856789,1505.260498
2,2,3,2018-08-21,CORP,IF,14774.066028,11638.146830
3,3,4,2018-07-05,RBR,DPA,2909.486234,1664.368925
4,4,5,2009-07-17,AG,EDPM,3765.942616,472.612518


**Nota:** La Norma local exige recolectar desde USD 1.000 (o USD 100 para cooperativas)

**Módulo 1 - Resumen del Dataset**

In [ ]:
# Resumen
total_loss_amount = (df_losses['Gross Loss Amount'] - df_losses['Recovery Amount']).sum()
total_events = len(df_losses)
avg_loss = total_loss_amount / total_events

print("Métricas Globales del Portafolio :")
print(f"Pérdida Neta Histórica Total: ${total_loss_amount:,.0f}")
print(f"Eventos de Pérdida Totales:    {total_events:,}")
print(f"Pérdida Neta Promedio:        ${avg_loss:,.0f}")

Métricas Globales del Portafolio :
Pérdida Neta Histórica Total: $30,163,728
Eventos de Pérdida Totales:    5,000
Pérdida Neta Promedio:        $6,033


In [ ]:
# Gráfico
loss_by_bl = df_losses.groupby('Business Line')[['Gross Loss Amount', 'Recovery Amount']].sum()
loss_by_bl['Net Loss'] = loss_by_bl['Gross Loss Amount'] - loss_by_bl['Recovery Amount']
loss_by_bl = loss_by_bl.sort_values(by='Net Loss', ascending=False)

fig = px.bar(
    loss_by_bl,
    x=loss_by_bl.index,
    y='Net Loss',
    title="Pérdida Neta Total por Línea de Negocio",
    labels={'Net Loss': 'Pérdida Neta (USD)', 'Business Line': 'Línea de Negocio'},
    template="plotly_white",
    width=800, height=400
)
fig.show()

**Módulo 2 - Análisis LDA**

Aquí usamos ipywidgets para crear el menú desplegable interactivo.

In [ ]:
# Función 'wrapper' para la interactividad
# Esta función se llamará cada vez que cambie el dropdown
def run_lda_analysis(business_line):
    # Se recomienda colocar este mensaje ya que el modelo se demora un poco ....
    print(f"Ejecutando Simulación de Monte Carlo para '{business_line}'...")
    (total_annual_losses,
     op_var_999,
     op_es_999,
     params) = lda_model(df_losses, business_line)

    if total_annual_losses is None:
        return # Error

    # Métricas de Capital
    print("\n Resultados de Capital: ")
    print("------------------------------")
    print(f"OpVaR (99.9%): ${op_var_999:,.0f}")
    print(f"OpES (99.9%):  ${op_es_999:,.0f}")
    print("-----------------------------\n")

    # Histograma
    plt.figure(figsize=(8, 3))
    sns.set_style("whitegrid")

    ax = sns.histplot(total_annual_losses, bins=30, stat="density", kde=False, label="Escenarios Simulados")

    # Añadir las líneas de VaR y ES
    ax.axvline(op_var_999, color="black", linestyle="--", linewidth=2,
               label=f"OpVaR 99.9% = ${op_var_999:,.0f}")

    ax.axvline(op_es_999, color="red", linestyle="--", linewidth=2,
               label=f"OpES 99.9% = ${op_es_999:,.0f}")

    # Configurar etiquetas y título
    ax.set_title(f"Distribución de Pérdidas Anuales ('{business_line}')")
    ax.set_xlabel("Pérdida Anual Total (USD)", fontsize=12)
    ax.set_ylabel("Densidad", fontsize=12)

    # Formatear el eje X para mostrar números completos (ej. 1,000,000 en lugar de 1e6)
    ax.xaxis.set_major_formatter(ticker.FuncFormatter(lambda x, p: format(int(x), ',')))
    plt.xticks(rotation=30)

    ax.legend()
    plt.show()
    # ------------------------------------------------
    # Mostrar Parámetros del Modelo
    print("\n Parámetros del Modelo Calibrado:")
    print("--------------------------------------------------------------------------------")
    print(f"  FRECUENCIA (Poisson) -> Lambda (Eventos/Año): {params[0]:.4f}")
    print(f"  SEVERIDAD (Log-Normal) -> Shape (s): {params[1]:.4f}, Loc: {params[2]:.4f}, Scale: {params[3]:.4f}")
    print("--------------------------------------------------------------------------------\n")

# Widget Interactivo
business_lines_options = df_losses['Business Line'].unique()
interact(
    run_lda_analysis,
    business_line=Dropdown(
        options=business_lines_options,
        value= 'CB',                      # Línea por defecto en el DashBoard
        description='Línea Negocio:'
    )
);

interactive(children=(Dropdown(description='Línea Negocio:', index=3, options=('CORP', 'RBR', 'AG', 'CB', 'PS'…

Indicadores claves de riesgo - KRIs
---

**Módulo 3 - Dashboard KRI**

Monitoreo proactivo de indicadores para la línea de


In [ ]:
# Datos específicos de KRIs de la entidad
data_kri = {
    'Fecha': [
        '30/11/2023', '31/12/2023', '31/01/2024', '29/02/2024', '31/03/2024', '30/04/2024',
        '31/05/2024', '30/06/2024', '31/07/2024', '31/08/2024', '30/09/2024', '31/10/2024',
        '30/11/2024', '31/12/2024', '31/01/2025', '28/02/2025', '31/03/2025', '30/04/2025',
        '31/05/2025', '30/06/2025', '31/07/2025', '31/08/2025', '30/09/2025', '31/10/2025'
    ],
    'Tasa de Rotación (%)': [
        8.5,   8.3, 10.2, 10.7, 10.2,  8.3,
        9.3,  11.2,  9.1, 10.9,  8.8,  9.8,
        9.3,  14.2, 10.3, 12.6,  9.9, 13.8,
       13.06, 14.2, 13.5, 15.8, 16.8, 12.2
    ],
    '% Parches Manuales': [
        2, 4, 3, 5,  5, 6,
        5, 8, 7, 6,  9, 9,
        8, 8, 9, 8,  9, 8,
       11, 9, 7, 13, 9, 11
    ],
    'Tickets Soporte TI (mes)': [
        25, 34, 27, 26, 32, 29,
        27, 20, 28, 25, 41, 32,
        38, 23, 31, 25, 25, 24,
        55, 33, 24, 34, 25, 31
    ]
}

df_kri = pd.DataFrame(data_kri)

# Columna 'Fecha' a formato datetime para los gráficos
df_kri['Fecha'] = pd.to_datetime(df_kri['Fecha'], format='%d/%m/%Y')
print("\nDatos de KRI:")
df_kri.head()


Datos de KRI:


,Fecha,Tasa de Rotación (%),% Parches Manuales,Tickets Soporte TI (mes)
0,2023-11-30,8.5,2,25
1,2023-12-31,8.3,4,34
2,2024-01-31,10.2,3,27
3,2024-02-29,10.7,5,26
4,2024-03-31,10.2,5,32


In [ ]:
# #df_kri = pd.read_excel('datakri.xlsx')
# print("\nDatos de KRI:")
# display(df_kri.head())

In [ ]:
# Tomamos el último dato como el "actual"
kri_actual = df_kri.iloc[-1]

# Umbrales (Semáforo) - Basado en la historia de la entidad
# Se deben validar con percentiles históricos o usar benchmarking sectorial

# Rotación
rot_verde = 10
rot_ambar = 14
# Parches
parch_verde = 8
parch_ambar = 12
# Tickets
tick_verde = 40
tick_ambar = 50

print("--- Indicadores de Alerta Temprana (Línea 'CB') - Estado Actual ")

# Gráfico de Velocímetro 1: Rotación
fig_rot = go.Figure(go.Indicator(
    mode="gauge+number",
    value=kri_actual['Tasa de Rotación (%)'],
    title={'text': "KRI Personas: Tasa de Rotación"},
    gauge={'axis': {'range': [0, 20]},
           'steps': [
                {'range': [0, rot_verde], 'color': "green"},
                {'range': [rot_verde, rot_ambar], 'color': "yellow"},
                {'range': [rot_ambar, 20], 'color': "red"}],
           'threshold': {'line': {'color': "black", 'width': 4}, 'thickness': 0.75, 'value': kri_actual['Tasa de Rotación (%)']}}
))
fig_rot.update_layout(height=250, margin=dict(l=30, r=30, t=60, b=20))
fig_rot.show()

--- Indicadores de Alerta Temprana (Línea 'CB') - Estado Actual 


In [ ]:
# Gráfico de Velocímetro 2: Parches Manuales
fig_parch = go.Figure(go.Indicator(
    mode="gauge+number",
    value=kri_actual['% Parches Manuales'],
    title={'text': "KRI Procesos: % Parches Manuales"},
    gauge={'axis': {'range': [0, 20]},
           'steps': [
                {'range': [0, parch_verde], 'color': "green"},
                {'range': [parch_verde, parch_ambar], 'color': "yellow"},
                {'range': [parch_ambar, 20], 'color': "red"}],
           'threshold': {'line': {'color': "black", 'width': 4}, 'thickness': 0.75, 'value': kri_actual['% Parches Manuales']}}
))
fig_parch.update_layout(height=250, margin=dict(l=30, r=30, t=60, b=20))
fig_parch.show()


In [ ]:
# Gráfico de Velocímetro 3: Tickets TI
fig_tick = go.Figure(go.Indicator(
    mode="gauge+number",
    value=kri_actual['Tickets Soporte TI (mes)'],
    title={'text': "KRI Sistemas: Tickets Soporte TI"},
    gauge={'axis': {'range': [0, 60]},
           'steps': [
                {'range': [0, tick_verde], 'color': "green"},
                {'range': [tick_verde, tick_ambar], 'color': "yellow"},
                {'range': [tick_ambar, 60], 'color': "red"}],
           'threshold': {'line': {'color': "black", 'width': 4}, 'thickness': 0.75, 'value': kri_actual['Tickets Soporte TI (mes)']}}
))
fig_tick.update_layout(height=250, margin=dict(l=30, r=30, t=60, b=20))
fig_tick.show()


In [ ]:
print("\n Evolución de KRIs (Últimos 24 Meses)")
# Gráficos de Tendencia
# KRI 1: Rotación
fig_rot_ts = px.line(df_kri, x='Fecha', y='Tasa de Rotación (%)', title='Evolución Tasa de Rotación',
                     width=800, height=400)
fig_rot_ts.add_hline(y=rot_verde, line_dash="dot", line_color="green")
fig_rot_ts.add_hline(y=rot_ambar, line_dash="dot", line_color="red", annotation_text="Límite Rojo", annotation_position="bottom right")
fig_rot_ts.show()


 Evolución de KRIs (Últimos 24 Meses)


In [ ]:
# KRI 2: Parches
fig_parch_ts = px.line(df_kri, x='Fecha', y='% Parches Manuales', title='Evolución % Parches Manuales',
                       width=800, height=400)
fig_parch_ts.add_hline(y=parch_verde, line_dash="dot", line_color="green")
fig_parch_ts.add_hline(y=parch_ambar, line_dash="dot", line_color="red", annotation_text="Límite Rojo", annotation_position="bottom right")
fig_parch_ts.show()

In [ ]:
# KRI 3: Tickets
fig_tick_ts = px.line(df_kri, x='Fecha', y='Tickets Soporte TI (mes)', title='Evolución Tickets Soporte TI',
                      width=800, height=400)
fig_tick_ts.add_hline(y=tick_verde, line_dash="dot", line_color="green")
fig_tick_ts.add_hline(y=tick_ambar, line_dash="dot", line_color="red", annotation_text="Límite Rojo", annotation_position="bottom right")
fig_tick_ts.show()

**Módulo 4 - Síntesis y Reporte**


**Integración con Capital (ICAAP)**: Proceso de Evaluación de la Adecuación del Capital Interno.

¿Están los KRIs en rojo? Sí: Lanza una alerta sugiriendo un "Capital Add-on" (Capital adicional).

El modelo LDA (Paso 2) mira el pasado (donde no hubo crisis). Los KRIs (Paso 3) dicen que el riesgo subió hoy. Por ende, el capital calculado por el modelo es insuficiente y se necesita un ajuste.


In [ ]:
linea = 'CB'
print(f" 1. Capital Modelo (LDA) -  Línea de negocio {linea} ")
(total_annual_losses,
 op_var_999,
 op_es_999,
 params) = lda_model(df_losses, linea) # Foco en 'CB'

if op_var_999:
    print(f"OpVaR (99.9%) Calculado: ${op_var_999:,.0f}")
    print(f"OpES (99.9%) Calculado:  ${op_es_999:,.0f}")
else:
    print(f"Error al calcular el modelo LDA para {linea}.")

print(f"\n 2. Estado Actual del Riesgo (KRIs) - linea de negocio {linea}")

# Umbrales definidos | Alternativa
rot_ambar = 14    # 10
parch_ambar = 12  # 10
tick_ambar = 50   # 30

# Chequeamos el estado
status_rot = "Rojo" if kri_actual['Tasa de Rotación (%)'] > rot_ambar else "Verde/Ámbar"
status_parch = "Rojo" if kri_actual['% Parches Manuales'] > parch_ambar else "Verde/Ámbar"
status_tick = "Rojo" if kri_actual['Tickets Soporte TI (mes)'] > tick_ambar else "Verde/Ámbar"

print(f"KRI Personas (Rotación): {kri_actual['Tasa de Rotación (%)']:.1f}%  -> Estado: {status_rot}")
print(f"KRI Procesos (Parches):  {kri_actual['% Parches Manuales']:.1f}% -> Estado: {status_parch}")
print(f"KRI Sistemas (Tickets):  {kri_actual['Tickets Soporte TI (mes)']:.0f}    -> Estado: {status_tick}")


 1. Capital Modelo (LDA) -  Línea de negocio CB 
OpVaR (99.9%) Calculado: $1,034,188
OpES (99.9%) Calculado:  $1,190,220

 2. Estado Actual del Riesgo (KRIs) - linea de negocio CB
KRI Personas (Rotación): 12.2%  -> Estado: Verde/Ámbar
KRI Procesos (Parches):  11.0% -> Estado: Verde/Ámbar
KRI Sistemas (Tickets):  31    -> Estado: Verde/Ámbar


In [ ]:
print("\n 3. Decisión: Ajuste de Capital (ICAAP)")
# Usamos display(en  HTML) para mostrar texto formateado (colores)
if status_rot == "Rojo" or status_parch == "Rojo" or status_tick == "Rojo":
    html_output = """
    <div style="background-color: #D32F2F; color: white; border-radius: 8px; padding: 20px; margin: 10px 0; font-family: sans-serif;">
    <h3 style="color: white; margin-top: 0; border-bottom: 1px solid #FFCDD2;"> !!!! ALERTA DE ESCENARIO: KRIs EN ROJO  !!! </h3>
    <p><strong>Observación:</strong> Múltiples KRIs (Personas, Procesos y/o Sistemas) están en 'Rojo'. Los datos de tendencia muestran un deterioro claro.</p>
    <p><strong>Implicación:</strong> El entorno de riesgo actual es <strong>peor</strong> que el promedio de los
    últimos 10 años. El modelo LDA (basado en el pasado) está <strong>subestimando</strong> la probabilidad de una gran pérdida futura.</p>
    <p><strong>Acción Recomendada (Reporte Regulatorio):</strong></p>
    <ol style="margin-left: 20px;">
        <li><strong>Activación de Análisis de Escenarios:</strong> Iniciar un taller con expertos de la línea
        para modelar el impacto de un "fallo catastrófico del sistema".</li>
        <li><strong>Ajuste de Capital (Capital Add-on):</strong> El capital regulatorio final debe ser
        el <strong>OpVaR del Modelo + un 'Add-on' de capital</strong> derivado del análisis de
        escenarios para reflejar el riesgo emergente.</li>
    </ol>
    </div>
    """
    display(HTML(html_output))
else:
    html_output = """
    <div style="background-color: #388E3C; color: white; border-radius: 8px; padding: 20px; margin: 10px 0; font-family: sans-serif;">
    <h3 style="color: white; margin-top: 0; border-bottom: 1px solid #C8E6C9;"> OK ... GESTIÓN NORMAL</h3>
    <p><strong>Observación:</strong> Los KRIs se encuentran dentro de los umbrales aceptables ('Verde' o 'Ámbar').</p>
    <p><strong>Implicación:</strong> El entorno de riesgo actual es consistente con el historial
    utilizado para calibrar el modelo LDA.</p>
    <p><strong>Acción Recomendada (Reporte Regulatorio):</strong></p>
    <ol style="margin-left: 20px;">
        <li><strong>Validación del Modelo:</strong> El capital calculado por el modelo LDA (OpVaR)
        se considera una estimación robusta y suficiente.</li>
        <li><strong>Monitoreo Continuo:</strong> Mantener la vigilancia sobre los KRIs.</li>
    </ol>
    </div>
    """
    display(HTML(html_output))


 3. Decisión: Ajuste de Capital (ICAAP)


**Recomendaciones**
---

El “Add-on” de capital cuando KRI está en rojo es cualitativo (o no cuantificado). Se pueden definir reglas cuantitativas (ej. +20% o +30% sobre OpVaR cuando 1 o 2 KRIs en rojo)

In [ ]:
# 3. Regla cuantitativa de Add-on
kri_score = 0
if status_rot == "Rojo": kri_score += 0.4
if status_parch == "Rojo": kri_score += 0.3
if status_tick == "Rojo": kri_score += 0.3

add_on_factor = 1 + kri_score * 0.7  # máximo +70% si los tres en rojo
capital_final = op_var_999 * add_on_factor
print(f"Capital con add-on KRI: ${capital_final:,.0f} (+{(add_on_factor-1)*100:.0f}%)")

Capital con add-on KRI: $1,034,188 (+0%)


**Escenarios de Estrés (Stress Testing)**

Tamien se puede agregar un choque sobre el parámetro lambda (frecuencia) en la simulación Monte Carlo multiplicándolo por (Ej.) 1.5 (que equivale a un shock del 50% en frecuencia debido a los KRIs rojos), para volver a calcular el VaR o ES.

**Contexto Regulatorio (SEPS / SB)**
---
La mayoría de entidades usan el Método Estándar (TSA), que es un porcentaje del Margen Bruto por línea de negocio. Pero se puede compare el Capital Económico (calculado por el modelo LDA) vs. el Capital Regulatorio (12-18% del Promedio de Ingresos Brutos de los últimos 3 años).

Esto genera alarmas que permiteb a la Gerencia tomar decisiones informdas: "La norma nos pide guardar Ej. $ \$10M$, pero nuestro modelo interno dice que el riesgo real es $ \$12M$. Por tanto, estamos descubiertos."


**Método TSA**

El TSA reconoce que hay negocios más riesgosos que otros. Divide al banco en 8 líneas de negocio estándar y asigna un factor de riesgo ($\beta$) a cada una:

* Alto Riesgo (18%): Finanzas Corporativas, Trading (Mesa de Dinero), Pagos y Liquidación.
* Riesgo Medio (15%): Banca Comercial, Servicios de Agencia.
* Riesgo Bajo (12%): Banca Minorista, Gestión de Activos, Intermediación Minorista.

El Capital Regulatorio es el promedio de los últimos 3 años de la suma ponderada de estos ingresos.


In [ ]:
# Datos de ingresos (IB) para el análisis TSA
data = {
    'Year': [2021, 2021, 2021, 2021, 2021, 2021,
             2022, 2022, 2022, 2022, 2022, 2022,
             2023, 2023, 2023, 2023, 2023, 2023],
    'Business Line': ['PS', 'AG', 'CB', 'RBR', 'CORP', 'TRD',
                      'PS', 'AG', 'CB', 'RBR', 'CORP', 'TRD',
                      'PS', 'AG', 'CB', 'RBR', 'CORP', 'TRD'],
    'Gross Income': [1.935891e+07, 1.332116e+07, 7.986884e+06, 4.843182e+06, 1.353559e+07, 1.792137e+07,
                     3.819639e+06, 1.883341e+07, 4.494359e+06, 1.344991e+07, 1.652478e+07, 9.394442e+06,
                     1.588702e+07, 1.634823e+07, 6.088136e+06, 6.485814e+06, 1.753726e+07, 2.196026e+06]
}

df_financials = pd.DataFrame(data)
print("Datos Financieros Cargados:")
df_financials.head()

Datos Financieros Cargados:


,Year,Business Line,Gross Income
0,2021,PS,19358910.0
1,2021,AG,13321160.0
2,2021,CB,7986884.0
3,2021,RBR,4843182.0
4,2021,CORP,13535590.0


In [ ]:
# df_financials[df_financials['Business Line'] == 'CB']

**Mapeo de Betas Regulatorios ($\beta$)**

Debemos asignar a cada una de tus líneas de negocio el porcentaje que exige el regulador (Basilea / Superintendencia).

In [ ]:
# Betas del enfoque TSA
# Mapeo según normativa estándar de Basilea

tsa_betas = {
    'CORP': 0.18,  # Corporate Finance
    'TRD':  0.18,  # Trading & Sales
    'PS':   0.18,  # Payment & Settlement
    'CB':   0.15,  # Commercial Banking
    'AG':   0.15,  # Agency Services
    'RBR':  0.12   # Retail Brokerage (o Retail Banking)
}

print("\n Betas asignados por Línea de Negocio:")
for bl, beta in tsa_betas.items():
    print(f"  {bl}: {beta*100:.0f}%")


 Betas asignados por Línea de Negocio:
  CORP: 18%
  TRD: 18%
  PS: 18%
  CB: 15%
  AG: 15%
  RBR: 12%


**Cálculo del Capital Regulatorio - TSA**

Ahora calculamos la carga de capital por año y luego promediamos los últimos 3 años.

In [ ]:
#  Cálculo de Capital Regulatorio (TSA)
def calculate_tsa_capital(df_income, betas):
    # 1. Asignar el Beta correspondiente a cada fila
    df_income['Beta'] = df_income['Business Line'].map(betas)

    # 2. Calcular el requerimiento por línea (Ingreso * Beta)
    df_income['Capital Charge'] = df_income['Gross Income'] * df_income['Beta']

    # 3. Sumar cargas por año (El TSA suma todas las líneas por año)
    annual_charges = df_income.groupby('Year')['Capital Charge'].sum()

    # 4. Promedio de los últimos 3 años
    regulatory_capital = annual_charges.mean()

    return regulatory_capital, annual_charges

reg_capital_tsa, annual_detail = calculate_tsa_capital(df_financials, tsa_betas)

print(f"\n Resultado Regulatorio: ")
print(f"Capital Regulatorio (TSA) TOtal: ${reg_capital_tsa:,.0f}")


 Resultado Regulatorio: 
Capital Regulatorio (TSA) Calculado: $11,315,267


**Integración y Análisis de Brecha (Gap Analysis)**

Vamos a comparar lo que dice tu modelo interno (LDA) vs. lo que exige la ley (TSA) específicamente para la línea que estamos analizando (por ejemplo, 'CB').

Nota: El TSA es un cálculo global del banco, pero para este ejercicio didáctico, vamos a aislar el requerimiento TSA solo de la línea 'CB' para compararlo manzanas con manzanas con el OpVaR de 'CB'.

In [ ]:
# Comparación Modelo Interno vs Regulatorio (Gap Analysis)
def compare_capital_models(business_line, op_var_999, df_income, betas):
    """
    Compara el Capital Económico (OpVaR) vs el Capital Regulatorio (TSA)
    para la línea de negocio seleccionada.
    """
    # 1. Calcular TSA específico para esta línea
    df_bl = df_income[df_income['Business Line'] == business_line].copy()
    df_bl['Capital Charge'] = df_bl['Gross Income'] * betas.get(business_line, 0.15)
    tsa_bl = df_bl.groupby('Year')['Capital Charge'].sum().mean()

    print(f"\n Análisis de Suficiencia de Capital para ({business_line}) ")
    print(f"1. Capital Económico (Tu Modelo LDA 99.9%): ${op_var_999:,.0f}")
    print(f"2. Capital Regulatorio (Norma TSA - Aprox):   ${tsa_bl:,.0f}")

    diff = op_var_999 - tsa_bl

    if diff > 0:
        html = f"""
        <div style='background-color: #FFF3E0; color: #E65100; padding: 15px; border-radius: 5px;'>
        <strong>ALERTA DE MODELO:</strong> Tu modelo interno sugiere un riesgo MAYOR (${diff:,.0f})
        al exigido por la norma. <br>
        Esto es positivo para la solvencia (eres prudente), pero costoso en eficiencia de capital.
        </div>
        """
    else:
        html = f"""
        <div style='background-color: #FFEBEE; color: #B71C1C; padding: 15px; border-radius: 5px;'>
        <strong>ALERTA DE SUFICIENCIA:</strong> Tu modelo interno está subestimando el riesgo
        en comparación con el estándar regulatorio (Déficit: ${abs(diff):,.0f}). <br>
        <strong>Acción:</strong> Revisa la calidad de tu data de pérdidas o ajusta los parámetros del LDA.
        </div>
        """
    display(HTML(html))

# COmparación de 'op_var_999' calculada previamente en el notebook)
# Asegúrate de haber corrido el modelo LDA para 'CB' antes de esto.
compare_capital_models('CB', op_var_999, df_financials, tsa_betas)


 Análisis de Suficiencia de Capital para (CB) 
1. Capital Económico (Tu Modelo LDA 99.9%): $1,034,188
2. Capital Regulatorio (Norma TSA - Aprox):   $928,469


**Limitaciones**:

 1 . *El modelo asume que las líneas de negocio son independientes*.

En una crisis sistémica (Ej. fallo de un proveedor de internet nacional), varias líneas fallarían a la vez. Sumar los VaR individuales sobreestima el capital (no hay beneficio de diversificación) o lo subestima si hay alta dependencia de cola.

In [ ]:
## End .......